Script to download "2016" (really 2018-2019 but using the images determined by the script that was run on the 2016 data) Malawi images

In [1]:
import os 
import pandas as pd
import numpy as np
import glob

In [2]:
download = False

In [3]:
!ls output/LSMS/malawi_2016/

candidate_download_locs.txt  consumptions.npy  households.npy  lons.npy
cluster.csv		     household.csv     lats.npy        nightlights.npy


In [4]:
locs = sorted(glob.glob('output/LSMS/*/candidate_download_locs.txt'))
locs
foldernames = [loc[12:loc.find('/',12)] for loc in locs]
fls = list(zip(foldernames,locs))
fls

[('malawi-2016', 'output/LSMS/malawi-2016/candidate_download_locs.txt'),
 ('malawi_2016', 'output/LSMS/malawi_2016/candidate_download_locs.txt'),
 ('nigeria_2013', 'output/LSMS/nigeria_2013/candidate_download_locs.txt'),
 ('tanzania_2011', 'output/LSMS/tanzania_2011/candidate_download_locs.txt'),
 ('uganda_2011', 'output/LSMS/uganda_2011/candidate_download_locs.txt')]

# Create Datasets

Each country in this folder should have:
1. 'nightlights.npy'
2. 'consumptions.npy'
These are aggregated at a cluster level.

This function will add in these values at the cluster level for each image. That is, we now get a dataframe that has cluster nightlight and consumption values for each image.

In [5]:
def create_df(country, df_orig):
    c_nightlight = np.load('output/LSMS/{}/nightlights.npy'.format(country))
    c_consumption = np.load('output/LSMS/{}/consumptions.npy'.format(country))
    c_groups = df_orig.groupby(['clust_lat', 'clust_lon'])
    counts = c_groups.count()
    counts['nightlight'] = c_nightlight
    counts['consumption'] = c_consumption
    counts = counts.reset_index().drop(['im_lat', 'im_lon'], axis=1)
    df_c = pd.merge(left=df_orig, right=counts, on=['clust_lat', 'clust_lon'])
    return df_c
dfs = [create_df(f,pd.read_csv(l, sep=' ', header=None, 
                               names=['im_lat', 'im_lon', 'clust_lat', 'clust_lon'])) for (f,l) in fls]
[df.shape for df in dfs]

[(90943, 6), (90943, 6), (67227, 6), (177931, 6), (74033, 6)]

In [6]:
# most nightlights are 0
# let's download images that have nonzero nightlights to induce variety into the model
for df in dfs:
    print((df['nightlight'] == 0).mean())
    print(df.shape)

0.4980042444168325
(90943, 6)
0.5004673256875185
(90943, 6)
0.2560132089785354
(67227, 6)
0.4047861249585514
(177931, 6)
0.5856982696905434
(74033, 6)


In [7]:
# Drop 10% of 0 nightlight images until  the percentages are less than 10
def drop_0s(df, frac=0.1):
    i = 1
    while (df['nightlight'].values==0).mean() > .1:
        i += 1
#         print(i)
        z_inds = np.argwhere(df['nightlight'].values == 0).reshape(-1)
        drop = np.random.choice(z_inds, int(frac*len(z_inds))).reshape(-1)
        df = df.drop(df.index[drop])
    return df
dfs = [drop_0s(df) for df in dfs]

In [8]:
[(df['nightlight'] < 1).mean() for df in dfs] # still most data is under 1 and small

[0.5017355632691701,
 0.49796,
 0.30389652736264333,
 0.3215225459189358,
 0.35740850059031876]

In [9]:
# The difference this made was only a few images so i didn't do this, quite arbitrary since the number of gridcells with
# nightlights below 1 in non-Malawi is quite low anyway.
def drop_under(df, cutoff=1, frac=0.1):
    z_inds = np.argwhere(df['nightlight'].values <= cutoff).reshape(-1)
    drop = np.random.choice(z_inds, int(frac*len(z_inds))).reshape(-1)
    df = df.drop(df.index[drop])
    return df

In [10]:
df = dfs[4]
from sklearn.mixture import GaussianMixture as GMM
X = df['nightlight'].values.reshape(-1,1)
gmm = GMM(n_components=3).fit(X)
labels = gmm.predict(df['nightlight'].values.reshape(-1,1))

In [11]:
(labels==0).mean(), (labels==1).mean(), (labels==2).mean()

(0.2641086186540732, 0.6264167650531287, 0.10947461629279812)

In [12]:
df['nightlight'][labels==0].max(), df['nightlight'][labels==1].max(), df['nightlight'][labels==2].max()

(59.30578512396694, 4.340909090909091, 25.380165289256198)

In [13]:
# let's smudge these numbers a little to raise the percentage in class 2
# we need the distribution to be somewhat even
# Mather did this for all, cutoffs seem to work pretty well for all countries.
(df['nightlight'] < 1.5).mean(), \
((df['nightlight'] >= 1.5) & (df['nightlight'] < 11)).mean(), \
(df['nightlight'] >= 11).mean()

(0.41685360094451, 0.28202479338842973, 0.3011216056670602)

In [14]:
d = []
for df in dfs:
    df['nightlight_bin'] = (df['nightlight'] < 1.5)*1 + \
                    ((df['nightlight'] >= 1.5) & (df['nightlight'] < 11))*2 + \
                    (df['nightlight'] >= 11)*3
    d.append(df)
dfs = d

In [15]:
for i in range(len(dfs)):
    dfs[i].to_csv('output/{}_guide.csv'.format(fls[i][0]), index = False)

In [16]:
# we don't need to download repeat images, as that reduces our download size significantly
# some images can belong to 2+ clusters
print(df.drop_duplicates(['im_lat', 'im_lon']).shape)

(25246, 7)


In [17]:
ll_download = [df.drop_duplicates(['im_lat', 'im_lon']) for df in dfs]
for i in range(len(ll_download)):
    ll_download[i].to_csv('output/{}_download.csv'.format(fls[i][0]), index=False)

In [18]:
# if the script fails halfway through, you can read this and see what is already downloaded
# remove the images already downloaded from this dataframe and then continue downloading

# mw_download = pd.read_csv('mw_2016_download.csv')
# downloaded = os.listdir('ims_malawi_2016/')
# lats = []
# longs = []
# for im in downloaded:
#     im = im[:-4].split('_')
#     lats.append(float(im[0]))
#     longs.append(float(im[1]))

# downloaded = pd.DataFrame.from_dict({'im_lat': lats, 'im_lon': longs})

# a = mw_download.set_index(['im_lat', 'im_lon']).index
# b = downloaded.set_index(['im_lat', 'im_lon']).index

# mask = ~a.isin(b)
# mw_download = mw_download.loc[mask].reset_index(drop=True)

# Download Images

Now we actually download images

In [19]:
"""Interface for downloading aerial imagery from Google Static Maps API.
- Get an API key at https://developers.google.com/maps/documentation/maps-static/intro
"""

import requests
from PIL import Image
from io import BytesIO

class ImageryDownloader:
    def __init__(self, access_token):
        """Initializes the object with an access token"""
        self.access_token = access_token
        self.url = 'https://maps.googleapis.com/maps/api/staticmap?center={},{}&zoom={}&size=400x400&maptype=satellite&key={}'
    
    def download(self, lat, long, zoom):
        """Downloads lat long
        """
        res = requests.get(self.url.format(lat, long, zoom, self.access_token))
        image = Image.open(BytesIO(res.content))

        return image

In [20]:
access = None
with open('api_key.txt', 'r') as f:
    access = f.readlines()[0]
    
im_downloader = ImageryDownloader(access)

In [21]:
for f,l in fls:
    os.makedirs('ims_{}'.format(f), exist_ok=True)
[df.shape for df in dfs],[ll.shape for ll in ll_download]

([(50704, 7), (50000, 7), (55203, 7), (116673, 7), (33880, 7)],
 [(28179, 7), (33580, 7), (46423, 7), (70772, 7), (25246, 7)])

In [ ]:
# Change selection to download
selection =4
f = fls[selection][0]
df = ll_download[selection]
print(f)
print('gathering images for im_lats and im_lons in this selection')
download = True
if download:
    im_names = []
    zoom = 16
    for i, r in df.iterrows():
        lat = r.im_lat
        long = r.im_lon
        try:
            im = im_downloader.download(lat, long, zoom)
            name = str(lat) + '_' + str(long)
            im.save('ims_{}/{}.png'.format(f,name))
            im_names.append(name + '.png')
        except:
            im_names.append(np.nan)
        if i % 100 == 0:
            # the counting is off because the indices from mw_download aren't continuous because we modified the dataframe
            print(i, end=', ')

    df['images'] = im_names
    df.to_csv('output/{}_download_info.csv'.format(f), index=False)

uganda_2011
gathering images for im_lats and im_lons in this selection
100, 